# Análisis de Sentimiento - Parte 2 (PCA)


Primero, bajemos los datos

#@title
!wget https://github.com/finiteautomata/imdb-dataset/raw/master/imdb_dataset.csv.zip
!unzip imdb_dataset.csv.zip

## Método de la Potencia

In [19]:
#@title
import numpy as np
from numpy import linalg as LA

def power_iteration(A, niter=1000, eps=1e-6):
    b = np.random.rand(A.shape[1])
    b = b / np.linalg.norm(b)

    for i in range(niter):
        old = b
        b = A @ b
        b = b / np.linalg.norm(b)

        # Criterio de parada
        # <a, b> = |a| |b| cos(angle)
        # -1 < cos(angle) < 1
        ## IMPORTANTE - criterio de parada adicional a niter > acelero el corte del método.
        ## Otros criterios de convergencia: angulo A*x ~ w*x
        cos_angle = np.dot(b, old)
        if (1 - eps) < cos_angle <= 1:
            print(f"Paré en la iteración {i+1}")
            break

    eigenvalue = np.dot(b, A @ b)

    return eigenvalue, b



In [20]:
#@title
D = np.diag(range(10, 0, -1))
print(D)
v = np.ones((D.shape[0], 1))
v = v / np.linalg.norm(v)

# Matriz de Householder
B = np.eye(D.shape[0]) - 2 * (v @ v.T)

# Matriz ya diagonalizada
M = D



[[10  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0]
 [ 0  0  0  0  0  0  4  0  0  0]
 [ 0  0  0  0  0  0  0  3  0  0]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  1]]


In [21]:
%%time
l, v = power_iteration(M, niter=1000, eps=1e-6)

Paré en la iteración 40
CPU times: user 4.2 ms, sys: 80 µs, total: 4.28 ms
Wall time: 3.28 ms


In [22]:
l, v

(9.99986781045605,
 array([9.99933904e-01, 1.14972725e-02, 3.36605776e-05, 8.02453497e-07,
        1.33465870e-09, 5.26925466e-13, 1.31081440e-17, 1.49197213e-21,
        1.24974473e-28, 1.03097185e-41]))

## Método de la potencia + deflación

In [23]:
def eigen(A, num, **kwargs):
    A = A.copy()
    eigenvalues = []
    eigenvectors = np.zeros((A.shape[0], num))
    
    for i in range(num):
        print(f"Autovalor {i+1}")
        l, v = power_iteration(A, **kwargs)
        eigenvalues.append(l)
        eigenvectors[:, i] = v
        A = A - l * np.outer(v, v)
        
    return np.array(eigenvalues), eigenvectors


In [24]:
N = 10
D = np.diag(range(N, 0, -1))

v = np.ones((D.shape[0], 1))
v = v / np.linalg.norm(v)

# Matriz de Householder
B = np.eye(D.shape[0]) - 2 * (v @ v.T)

M = B.T @ D @ B
# Sobre la misma de antes....ahora para todos...
l, v = eigen(M, N, niter=5_000)
print(l)
print(v)
print(B)

Autovalor 1
Paré en la iteración 34
Autovalor 2
Paré en la iteración 25
Autovalor 3
Paré en la iteración 34
Autovalor 4
Paré en la iteración 37
Autovalor 5
Paré en la iteración 25
Autovalor 6
Paré en la iteración 26
Autovalor 7
Paré en la iteración 19
Autovalor 8
Paré en la iteración 17
Autovalor 9
Paré en la iteración 11
Autovalor 10
Paré en la iteración 2
[9.9998661  9.00005767 7.99999813 7.00006467 6.00001768 5.0000298
 4.00001574 3.00001711 2.00000506 1.0000028 ]
[[-0.7975438   0.21250998  0.20023806  0.19865013  0.1996453   0.19938435
   0.19952618  0.1993495   0.19961553  0.19952794]
 [ 0.19083574 -0.80020658  0.20989759  0.20154466  0.19969952  0.19938412
   0.19952618  0.1993495   0.19961553  0.19952794]
 [ 0.20193176  0.19071903 -0.80010671  0.20972649  0.19953195  0.19938557
   0.19952618  0.1993495   0.19961553  0.19952794]
 [ 0.20238697  0.19736145  0.19016088 -0.80121803  0.20826605  0.19931931
   0.19952576  0.19934951  0.19961553  0.19952794]
 [ 0.20238933  0.19961624  0

In [ ]:
#@title
A = np.array([
  [7, 2, 3],
  [0, 2, 0],
  [-6, -2, -2]
])


w, V = eigen(A, num=3, niter=20000, eps=1e-24)
print("w")
print(w)
print("V")
print(V)

In [ ]:
print(A)
for i in range(len(A)):
    print(i)
    print(np.allclose(A @ V[:, i], w[i] * V[:,i]))

In [ ]:
print("uso una biblioteca")
w2, V2 = LA.eig(A)
print(w2, "\n", V2)
print(np.dot(V2[:,0],V2[:,2]))
print(np.dot(V2[:,0],V2[:,1]))
print(np.dot(V2[:,2],V2[:,1])) 
# no son ortogonales (repasar pdf teórico método potencia)

In [ ]:
#@title
A = np.array([
  [ 7,  2,  -3],
  [ 2,  2,  -2],
  [-3, -2,  -2]
])
w, V = eigen(A, num=3, niter=20000, eps=1e-24)
for i in range(len(A)):
    print(i)
    print(np.allclose(A @ V[:, i], w[i] * V[:,i]))
print("Gracias teorema espectral! (ver pdf teórico)")  

In [ ]:
#@title

AT = A @ A.T
TA = A.T @ A
wta, VTA = eigen(AT, num=3, niter=20000, eps=1e-24)
wat, VAT = eigen(TA, num=3, niter=20000, eps=1e-24)
print("Bingo :)" if np.allclose(wat,wta) else "Bongo :(")


## Volvamos a Análisis de Sentimiento, aplicando PCA

---



In [25]:
#@title
import pandas as pd 
import sklearn

df = pd.read_csv("IMDB Dataset.csv")
df = df.sample(frac=1, random_state=2020)

df_train = df[:10000]
df_test = df[10000:13000]

text_train, text_test = df_train["review"], df_test["review"]
label_train, label_test = df_train["sentiment"], df_test["sentiment"]

print("Class balance : {} pos {} neg".format(
    (label_train == 'positive').sum() / label_train.shape[0], 
    (label_train == 'negative').sum() / label_train.shape[0]
))
print("Cantidad de documentos: {}".format(df.shape[0]))

Class balance : 0.5007 pos 0.4993 neg
Cantidad de documentos: 50000


label train & test


In [26]:
#@title
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=3, max_features=5000, binary=True)
# Convert a collection of text documents to a matrix of token counts.

vect.fit(text_train)

X_train = vect.transform(text_train)
X_test = vect.transform(text_test)

y_train = label_train# == 'positive' # Convertimos a vectores booleanos
y_test = label_test# == "positive"

## PCA

Vamos a ver una técnica para reducir la dimensionalidad y aún así mantener la mayor cantidad de información posible.

Recordemos que, dada X su matriz de covarianza $M_X$ es

$$ M_X = \frac{X^T X}{n-1} $$

En primer lugar, calculemos la matriz de covarianza de $X_{train}$

In [27]:
#@title
import numpy as np

# Esto es porque me lo convierte a np.matrix si no
X = np.array(X_train - X_train.mean(axis=0))

cov_matrix = X.T @ X / (X.shape[0]-1) 

In [28]:
#@title
#time .... 3 mins
w, V = eigen(cov_matrix, num=50, niter=1000, eps=1e-6)

Autovalor 1
Paré en la iteración 5
Autovalor 2
Paré en la iteración 9
Autovalor 3
Paré en la iteración 173
Autovalor 4
Paré en la iteración 54
Autovalor 5
Paré en la iteración 58
Autovalor 6
Paré en la iteración 61
Autovalor 7
Paré en la iteración 84
Autovalor 8
Paré en la iteración 48
Autovalor 9
Paré en la iteración 348
Autovalor 10
Paré en la iteración 119
Autovalor 11
Paré en la iteración 61
Autovalor 12
Paré en la iteración 84
Autovalor 13
Paré en la iteración 145
Autovalor 14
Paré en la iteración 125
Autovalor 15
Paré en la iteración 153
Autovalor 16
Paré en la iteración 129
Autovalor 17
Paré en la iteración 299
Autovalor 18
Paré en la iteración 127
Autovalor 19
Paré en la iteración 211
Autovalor 20
Paré en la iteración 171
Autovalor 21
Paré en la iteración 196
Autovalor 22
Paré en la iteración 198
Autovalor 23
Paré en la iteración 209
Autovalor 24
Paré en la iteración 523
Autovalor 25
Paré en la iteración 262
Autovalor 26
Paré en la iteración 150
Autovalor 27
Paré en la iteració

A ver los autovalores...

In [29]:
#@title
import matplotlib.pyplot as plt

plt.plot(w)
plt.xlabel("# lambda")
plt.ylabel("lambda")

Text(0, 0.5, 'lambda')

## Recordando cambios de base y otros yuyos

Sup $M_X$ es la matriz de la covarianza, 

$$ M_X = \frac{X^T X}{n-1} $$

Si $B = \{v_1, \ldots , v_n\}$ es la base ortogonal de autovectores de $M_X$ entonces la matriz cambio de base de $B$ a la base canónica ($E$) se escribe como la matriz cuyas columnas son los respectivos vectores

$$C_{B, E} = V = \begin{bmatrix}
        &     & \ldots &     \\
    v_1 & v_2 & \ldots & v_n \\
        &     & \ldots &     \\
\end{bmatrix}
$$


La matriz inversa de ésta es la cambio de base de $E$ a $B$. Es decir $C_{E, B} = C_{B, E}^{-1}$. Como nuestra base es ortogonal, tenemos

$$ C_{E, B} = C_{B, E}^T = V^T = \begin{bmatrix}
& & v_1   & &\\
& & v_2   & &\\
& &\vdots & & \\
& & v_n   & & \\
\end{bmatrix}
$$

Es decir, la matriz que consiste de apilar los vectores fila de la base $B$.


### Cambiando de base nuestras instancias de entrenamiento

Nuestras matrices $X \in R^{n \times m}$ con $n$ igual a la cantidad de instancias de entrenamiento, y $m$ la cantidad de variables

Tenemos entonces

$$ X = \begin{bmatrix}
& & x^{(1)}   & &\\
& & x^{(2)}   & &\\
& &\vdots & & \\
& & v_n   & & \\
\end{bmatrix}
$$

Si $x$ es una instancia de entrenamiento a la cual queremos cambiar de base, queremos hacer

$$\overline x = V^T x$$

Luego, si queremos cambiar de base cada instancia, hacemos...

$$ V^T X^T = V^T \begin{bmatrix}
        & &    & &\\
x^{(1)}& x^{(2)} &\ldots & x ^{(n-1)} & x ^{(n)} \\
& & & & \\
\end{bmatrix} = \begin{bmatrix}
        & &    & &\\
V^T x^{(1)}& V^T x^{(2)} &\ldots & V^T x ^{(n-1)} & V^T x ^{(n)} \\
& & & & \\
\end{bmatrix} = \begin{bmatrix}
        & &    & &\\
\overline{x^{(1)}}& \overline{x^{(2)}} &\ldots & \overline{x^{(n-1)}} & \overline{x^{(n)}} \\
& & & & \\
\end{bmatrix}
$$

Ahora, lo que necesitamos es que cada instancia esté en una fila, así que trasponemos

$$
\overline{X} = (V^T X^T)^T = X V
$$

## Ejercicio:

1. Implementar el cambio de base usando su implementación del método de la potencia
2. Experimentar con distintos $\alpha$. ¿Cómo afecta la accuracy de nuestro algoritmo? 
3. Para experimentar en casa: ¿Es más rápido?

In [ ]:
# reeplazar por ejercicio peer question

In [ ]:
#@title
X_pca_train = X_train @ V
X_pca_test = X_test @ V

X_pca_train.shape

Entrenar ahora el clasificador usando estas nuevas instancias[]

In [ ]:
#@title
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=25)
clf.fit(X_pca_train, y_train)
# sólo seteamos 25 vecinos, resto default (que se ve en el output)
# ‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.
# 'algorithm' Algorithm used to compute the nearest neighbors.
# 'leaf_size'  - Leaf size passed to BallTree or KDTree (posibles algos)
#  metric -- > métrica a usar
# Power parameter for the Minkowski metric. 
# When p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.
# n_jobs The number of parallel jobs to run for neighbors search.

In [ ]:
#@title
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_pca_test)
### Accuracy:
### "lo bueno" (true positive + true negative) / "todo" (true positive + true negative + false positive + false negative)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

In [ ]:
#@title
from tqdm.auto import tqdm
alphas = range(2, 50, 2)
results = []
# Hacemos varias corridas recortando los V de PCA.
# tqdm para tener el progress meter.
for alpha in tqdm(alphas):
    #print(alpha)
    X_pca_train = X_train @ V[:, :alpha]
    X_pca_test = X_test @ V[:, :alpha]

    clf = KNeighborsClassifier(n_neighbors=25)
    clf.fit(X_pca_train, y_train)

    y_pred = clf.predict(X_pca_test)
    acc = accuracy_score(y_test, y_pred)
    results.append(acc)

In [ ]:
#@title
import matplotlib.pyplot as plt

plt.plot(alphas, results, "o-")
plt.xlabel("# PCs")
plt.ylabel("Accuracy")
## Comparar con la solución sin PCA.

In [ ]:
#@title
